# Time Series Forecasting for Portfolio Management Optimization
## GMF Investments - Financial Analysis Challenge

**Analyst:** [Your Name]  
**Date:** August 2025  
**Period:** July 1, 2015 - July 31, 2025  

### Executive Summary
This analysis implements time series forecasting models to optimize portfolio allocation across three key assets: Tesla (TSLA), Vanguard Total Bond Market ETF (BND), and S&P 500 ETF (SPY). We develop both statistical (ARIMA) and deep learning (LSTM) models to forecast future market trends and apply Modern Portfolio Theory for optimization.

In [ ]:
# Import required libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import scipy.optimize as sco
from scipy import stats

# Time series and forecasting
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima

# Deep learning
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

# Portfolio optimization
import pypfopt
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Suppress warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Libraries imported successfully!")

## Task 1: Data Preprocessing and Exploration

### 1.1 Data Collection

In [ ]:
# Define the assets and time period
assets = ['TSLA', 'BND', 'SPY']
start_date = '2015-07-01'
end_date = '2025-07-31'

# Download data
print("Downloading financial data...")
data = {}
for asset in assets:
    try:
        ticker = yf.Ticker(asset)
        data[asset] = ticker.history(start=start_date, end=end_date)
        print(f"✓ {asset}: {len(data[asset])} records downloaded")
    except Exception as e:
        print(f"✗ Error downloading {asset}: {e}")

# Create a combined dataframe with adjusted close prices
price_data = pd.DataFrame()
for asset in assets:
    if asset in data:
        price_data[asset] = data[asset]['Close']

print(f"\nCombined dataset shape: {price_data.shape}")
print(f"Date range: {price_data.index.min()} to {price_data.index.max()}")

### 1.2 Data Cleaning and Basic Statistics

In [ ]:
# Check for missing values
print("Missing values per asset:")
print(price_data.isnull().sum())

# Handle missing values by forward filling
price_data = price_data.fillna(method='ffill').dropna()

# Basic statistics
print("\nBasic Statistics:")
print(price_data.describe())

# Data types
print("\nData Types:")
print(price_data.dtypes)

### 1.3 Calculate Returns and Risk Metrics

In [ ]:
# Calculate daily returns
returns = price_data.pct_change().dropna()

# Calculate rolling volatility (30-day window)
rolling_vol = returns.rolling(window=30).std() * np.sqrt(252)  # Annualized

# Calculate cumulative returns
cumulative_returns = (1 + returns).cumprod()

print("Returns Statistics:")
print(returns.describe())

# Calculate key risk metrics
def calculate_risk_metrics(returns_series, confidence_level=0.05):
    """
    Calculate Value at Risk (VaR) and Sharpe Ratio
    """
    # VaR calculation
    var = np.percentile(returns_series, confidence_level * 100)
    
    # Sharpe Ratio (assuming risk-free rate of 2%)
    risk_free_rate = 0.02 / 252  # Daily risk-free rate
    excess_returns = returns_series - risk_free_rate
    sharpe_ratio = excess_returns.mean() / returns_series.std() * np.sqrt(252)
    
    return var, sharpe_ratio

print("\nRisk Metrics:")
for asset in assets:
    var, sharpe = calculate_risk_metrics(returns[asset])
    print(f"{asset}: VaR (5%) = {var:.4f}, Sharpe Ratio = {sharpe:.4f}")

### 1.4 Exploratory Data Analysis

In [ ]:
# Create comprehensive EDA plots
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
fig.suptitle('Comprehensive Financial Data Analysis', fontsize=16, fontweight='bold')

# 1. Price Evolution
ax1 = axes[0, 0]
for asset in assets:
    ax1.plot(price_data.index, price_data[asset], label=asset, linewidth=1.5)
ax1.set_title('Price Evolution Over Time')
ax1.set_ylabel('Price ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Normalized Price Evolution
ax2 = axes[0, 1]
normalized_prices = price_data / price_data.iloc[0]
for asset in assets:
    ax2.plot(normalized_prices.index, normalized_prices[asset], label=asset, linewidth=1.5)
ax2.set_title('Normalized Price Evolution (Base = 1)')
ax2.set_ylabel('Normalized Price')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Daily Returns Distribution
ax3 = axes[1, 0]
returns['TSLA'].hist(bins=50, alpha=0.7, ax=ax3, color='red')
ax3.set_title('TSLA Daily Returns Distribution')
ax3.set_xlabel('Daily Returns')
ax3.set_ylabel('Frequency')
ax3.axvline(returns['TSLA'].mean(), color='black', linestyle='--', label=f'Mean: {returns["TSLA"].mean():.4f}')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Rolling Volatility
ax4 = axes[1, 1]
ax4.plot(rolling_vol.index, rolling_vol['TSLA'], label='TSLA', color='red', linewidth=1.5)
ax4.set_title('TSLA Rolling Volatility (30-day)')
ax4.set_ylabel('Annualized Volatility')
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Correlation Heatmap
ax5 = axes[2, 0]
correlation_matrix = returns.corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=ax5)
ax5.set_title('Asset Correlation Matrix')

# 6. Cumulative Returns
ax6 = axes[2, 1]
for asset in assets:
    ax6.plot(cumulative_returns.index, cumulative_returns[asset], label=asset, linewidth=1.5)
ax6.set_title('Cumulative Returns')
ax6.set_ylabel('Cumulative Return')
ax6.legend()
ax6.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 1.5 Stationarity Testing

In [ ]:
def adf_test(series, title=''):
    """
    Perform Augmented Dickey-Fuller test for stationarity
    """
    result = adfuller(series.dropna())
    print(f'\n{title}')
    print(f'ADF Statistic: {result[0]:.6f}')
    print(f'p-value: {result[1]:.6f}')
    print(f'Critical Values:')
    for key, value in result[4].items():
        print(f'\t{key}: {value:.3f}')
    
    if result[1] <= 0.05:
        print("✓ Series is stationary (reject null hypothesis)")
        return True
    else:
        print("✗ Series is non-stationary (fail to reject null hypothesis)")
        return False

print("STATIONARITY TESTING")
print("=" * 50)

# Test stationarity for prices
for asset in assets:
    adf_test(price_data[asset], f'{asset} Price Series')

print("\n" + "=" * 50)
print("TESTING RETURNS (FIRST DIFFERENCE)")
print("=" * 50)

# Test stationarity for returns
for asset in assets:
    adf_test(returns[asset], f'{asset} Returns Series')

### 1.6 Outlier Detection

In [ ]:
# Detect outliers using IQR method
def detect_outliers(series, multiplier=1.5):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR
    
    outliers = series[(series < lower_bound) | (series > upper_bound)]
    return outliers

print("OUTLIER ANALYSIS")
print("=" * 50)

for asset in assets:
    outliers = detect_outliers(returns[asset])
    print(f"\n{asset}:")
    print(f"Number of outliers: {len(outliers)}")
    print(f"Percentage of outliers: {len(outliers)/len(returns[asset])*100:.2f}%")
    
    if len(outliers) > 0:
        print(f"Extreme outliers:")
        extreme_outliers = outliers.nlargest(3).append(outliers.nsmallest(3))
        for date, value in extreme_outliers.items():
            print(f"  {date.strftime('%Y-%m-%d')}: {value:.4f}")

## Task 2: Time Series Forecasting Models

### 2.1 Data Preparation for Modeling

In [ ]:
# Focus on TSLA for forecasting
tsla_prices = price_data['TSLA'].copy()
tsla_returns = returns['TSLA'].copy()

# Split data chronologically
split_date = '2024-01-01'
train_prices = tsla_prices[tsla_prices.index < split_date]
test_prices = tsla_prices[tsla_prices.index >= split_date]

train_returns = tsla_returns[tsla_returns.index < split_date]
test_returns = tsla_returns[tsla_returns.index >= split_date]

print(f"Training set: {len(train_prices)} observations")
print(f"Test set: {len(test_prices)} observations")
print(f"Training period: {train_prices.index.min()} to {train_prices.index.max()}")
print(f"Test period: {test_prices.index.min()} to {test_prices.index.max()}")

### 2.2 ARIMA Model Implementation

In [ ]:
# Auto ARIMA for optimal parameters
print("Finding optimal ARIMA parameters...")
auto_model = auto_arima(train_returns, 
                       start_p=0, start_q=0, 
                       max_p=5, max_q=5, 
                       seasonal=False, 
                       stepwise=True, 
                       suppress_warnings=True,
                       error_action='ignore')

print(f"Optimal ARIMA order: {auto_model.order}")
print(auto_model.summary())

# Fit ARIMA model
arima_model = ARIMA(train_returns, order=auto_model.order)
arima_fitted = arima_model.fit()

# Generate forecasts
n_periods = len(test_returns)
arima_forecast = arima_fitted.forecast(steps=n_periods)
arima_conf_int = arima_fitted.get_forecast(steps=n_periods).conf_int()

print(f"\nARIMA forecast generated for {n_periods} periods")

### 2.3 LSTM Model Implementation

In [ ]:
# Prepare data for LSTM
def create_lstm_dataset(data, look_back=60):
    """
    Create dataset for LSTM with look_back window
    """
    X, y = [], []
    for i in range(look_back, len(data)):
        X.append(data[i-look_back:i])
        y.append(data[i])
    return np.array(X), np.array(y)

# Scale the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(train_prices.values.reshape(-1, 1))

# Create LSTM dataset
look_back = 60
X_train, y_train = create_lstm_dataset(scaled_prices.flatten(), look_back)

# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))

print(f"LSTM training data shape: {X_train.shape}")
print(f"LSTM target data shape: {y_train.shape}")

# Build LSTM model
lstm_model = Sequential([
    LSTM(50, return_sequences=True, input_shape=(look_back, 1)),
    Dropout(0.2),
    LSTM(50, return_sequences=False),
    Dropout(0.2),
    Dense(25),
    Dense(1)
])

lstm_model.compile(optimizer='adam', loss='mean_squared_error')
print("LSTM model architecture:")
lstm_model.summary()

In [ ]:
# Train LSTM model
print("Training LSTM model...")
history = lstm_model.fit(X_train, y_train, 
                        batch_size=32, 
                        epochs=50, 
                        validation_split=0.1,
                        verbose=1)

# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Generate LSTM predictions
# Prepare test data
all_prices = pd.concat([train_prices, test_prices])
scaled_all_prices = scaler.fit_transform(all_prices.values.reshape(-1, 1))

# Create test dataset
test_start_idx = len(train_prices) - look_back
test_data = scaled_all_prices[test_start_idx:]

X_test = []
for i in range(look_back, len(test_data)):
    X_test.append(test_data[i-look_back:i])

X_test = np.array(X_test)
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Make predictions
lstm_predictions_scaled = lstm_model.predict(X_test)
lstm_predictions = scaler.inverse_transform(lstm_predictions_scaled)

print(f"LSTM predictions generated: {len(lstm_predictions)} values")

### 2.4 Model Evaluation and Comparison

In [ ]:
# Convert ARIMA return forecasts to price forecasts
arima_price_forecast = []
last_price = train_prices.iloc[-1]

for return_forecast in arima_forecast:
    next_price = last_price * (1 + return_forecast)
    arima_price_forecast.append(next_price)
    last_price = next_price

arima_price_forecast = np.array(arima_price_forecast)

# Calculate evaluation metrics
def calculate_metrics(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mae, rmse, mape

# Ensure same length for comparison
min_length = min(len(test_prices), len(arima_price_forecast), len(lstm_predictions))
actual_prices = test_prices.iloc[:min_length].values
arima_pred = arima_price_forecast[:min_length]
lstm_pred = lstm_predictions[:min_length].flatten()

# Calculate metrics
arima_mae, arima_rmse, arima_mape = calculate_metrics(actual_prices, arima_pred)
lstm_mae, lstm_rmse, lstm_mape = calculate_metrics(actual_prices, lstm_pred)

# Results comparison
results_df = pd.DataFrame({
    'Model': ['ARIMA', 'LSTM'],
    'MAE': [arima_mae, lstm_mae],
    'RMSE': [arima_rmse, lstm_rmse],
    'MAPE (%)': [arima_mape, lstm_mape]
})

print("MODEL PERFORMANCE COMPARISON")
print("=" * 50)
print(results_df.to_string(index=False))

# Determine best model
best_model = 'ARIMA' if arima_rmse < lstm_rmse else 'LSTM'
print(f"\nBest performing model: {best_model}")

In [ ]:
# Visualization of predictions
plt.figure(figsize=(15, 8))

# Plot historical data
plt.plot(train_prices.index[-200:], train_prices.iloc[-200:], 
         label='Historical Prices', color='blue', linewidth=2)

# Plot actual test prices
plt.plot(test_prices.index[:min_length], actual_prices, 
         label='Actual Prices', color='black', linewidth=2)

# Plot predictions
plt.plot(test_prices.index[:min_length], arima_pred, 
         label=f'ARIMA Forecast (RMSE: {arima_rmse:.2f})', 
         color='red', linewidth=2, linestyle='--')

plt.plot(test_prices.index[:min_length], lstm_pred, 
         label=f'LSTM Forecast (RMSE: {lstm_rmse:.2f})', 
         color='green', linewidth=2, linestyle='-.')

plt.title('TSLA Price Forecasting: Model Comparison', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Task 3: Future Market Trends Forecasting

### 3.1 Generate 6-12 Month Forecasts

In [ ]:
# Generate future forecasts (6-12 months)
forecast_periods = 252  # Approximately 12 months of trading days
future_dates = pd.date_range(start=tsla_prices.index[-1] + timedelta(days=1), 
                           periods=forecast_periods, freq='B')

# Use the best performing model for future forecasts
if best_model == 'ARIMA':
    # Refit ARIMA on full dataset
    full_arima = ARIMA(tsla_returns, order=auto_model.order)
    full_arima_fitted = full_arima.fit()
    
    # Generate future return forecasts
    future_returns_forecast = full_arima_fitted.forecast(steps=forecast_periods)
    future_conf_int = full_arima_fitted.get_forecast(steps=forecast_periods).conf_int()
    
    # Convert to price forecasts
    future_prices_forecast = []
    future_prices_lower = []
    future_prices_upper = []
    
    last_price = tsla_prices.iloc[-1]
    
    for i, return_forecast in enumerate(future_returns_forecast):
        # Point forecast
        next_price = last_price * (1 + return_forecast)
        future_prices_forecast.append(next_price)
        
        # Confidence intervals
        lower_return = future_conf_int.iloc[i, 0]
        upper_return = future_conf_int.iloc[i, 1]
        
        future_prices_lower.append(last_price * (1 + lower_return))
        future_prices_upper.append(last_price * (1 + upper_return))
        
        last_price = next_price
    
    future_prices_forecast = np.array(future_prices_forecast)
    future_prices_lower = np.array(future_prices_lower)
    future_prices_upper = np.array(future_prices_upper)
    
else:
    # Use LSTM for future forecasts
    # This is a simplified approach - in practice, you'd want to retrain on full data
    future_prices_forecast = lstm_pred  # Placeholder
    future_prices_lower = future_prices_forecast * 0.9  # Simple confidence interval
    future_prices_upper = future_prices_forecast * 1.1

print(f"Generated {forecast_periods} future price forecasts using {best_model} model")
print(f"Forecast period: {future_dates[0]} to {future_dates[-1]}")

### 3.2 Forecast Visualization and Analysis

In [ ]:
# Create comprehensive forecast visualization
plt.figure(figsize=(16, 10))

# Historical data (last 2 years)
historical_cutoff = tsla_prices.index[-500:]
plt.plot(historical_cutoff, tsla_prices.loc[historical_cutoff], 
         label='Historical Prices', color='blue', linewidth=2)

# Future forecasts
plt.plot(future_dates, future_prices_forecast, 
         label=f'{best_model} Forecast', color='red', linewidth=2, linestyle='--')

# Confidence intervals
plt.fill_between(future_dates, future_prices_lower, future_prices_upper, 
                alpha=0.3, color='red', label='95% Confidence Interval')

plt.title('TSLA 12-Month Price Forecast', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Price ($)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Forecast statistics
current_price = tsla_prices.iloc[-1]
forecast_end_price = future_prices_forecast[-1]
total_return = (forecast_end_price - current_price) / current_price * 100

print("\nFORECAST ANALYSIS")
print("=" * 50)
print(f"Current TSLA Price: ${current_price:.2f}")
print(f"12-Month Forecast: ${forecast_end_price:.2f}")
print(f"Expected Total Return: {total_return:.2f}%")
print(f"Annualized Return: {total_return:.2f}%")

# Confidence interval analysis
ci_width_start = future_prices_upper[0] - future_prices_lower[0]
ci_width_end = future_prices_upper[-1] - future_prices_lower[-1]
ci_expansion = (ci_width_end - ci_width_start) / ci_width_start * 100

print(f"\nCONFIDENCE INTERVAL ANALYSIS")
print(f"Initial CI Width: ${ci_width_start:.2f}")
print(f"Final CI Width: ${ci_width_end:.2f}")
print(f"CI Expansion: {ci_expansion:.1f}%")
print("\nImplication: Uncertainty increases significantly over longer forecast horizons")

## Task 4: Portfolio Optimization

### 4.1 Expected Returns Calculation

In [ ]:
# Calculate expected returns
# TSLA: Use forecast-based expected return
tsla_expected_return = total_return / 100  # Convert percentage to decimal

# BND and SPY: Use historical average returns
historical_returns_annual = returns.mean() * 252
bnd_expected_return = historical_returns_annual['BND']
spy_expected_return = historical_returns_annual['SPY']

# Create expected returns vector
expected_returns_vector = pd.Series({
    'TSLA': tsla_expected_return,
    'BND': bnd_expected_return,
    'SPY': spy_expected_return
})

print("EXPECTED RETURNS (ANNUALIZED)")
print("=" * 40)
for asset, ret in expected_returns_vector.items():
    print(f"{asset}: {ret:.4f} ({ret*100:.2f}%)")

# Calculate covariance matrix
cov_matrix = returns.cov() * 252  # Annualized

print("\nCOVARIANCE MATRIX (ANNUALIZED)")
print("=" * 40)
print(cov_matrix)

### 4.2 Efficient Frontier Generation

In [ ]:
# Generate Efficient Frontier
def generate_efficient_frontier(expected_returns, cov_matrix, num_portfolios=10000):
    """
    Generate efficient frontier using Monte Carlo simulation
    """
    num_assets = len(expected_returns)
    results = np.zeros((3, num_portfolios))
    weights_array = np.zeros((num_portfolios, num_assets))
    
    # Risk-free rate (2% annual)
    risk_free_rate = 0.02
    
    for i in range(num_portfolios):
        # Generate random weights
        weights = np.random.random(num_assets)
        weights /= np.sum(weights)  # Normalize to sum to 1
        weights_array[i] = weights
        
        # Calculate portfolio return and volatility
        portfolio_return = np.sum(weights * expected_returns)
        portfolio_volatility = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights)))
        
        # Calculate Sharpe ratio
        sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_volatility
        
        # Store results
        results[0, i] = portfolio_return
        results[1, i] = portfolio_volatility
        results[2, i] = sharpe_ratio
    
    return results, weights_array

# Generate efficient frontier
print("Generating Efficient Frontier...")
results, weights = generate_efficient_frontier(expected_returns_vector, cov_matrix)

# Find optimal portfolios
max_sharpe_idx = np.argmax(results[2])
min_vol_idx = np.argmin(results[1])

# Maximum Sharpe Ratio Portfolio
max_sharpe_return = results[0, max_sharpe_idx]
max_sharpe_volatility = results[1, max_sharpe_idx]
max_sharpe_ratio = results[2, max_sharpe_idx]
max_sharpe_weights = weights[max_sharpe_idx]

# Minimum Volatility Portfolio
min_vol_return = results[0, min_vol_idx]
min_vol_volatility = results[1, min_vol_idx]
min_vol_ratio = results[2, min_vol_idx]
min_vol_weights = weights[min_vol_idx]

print("Efficient Frontier generated successfully!")

### 4.3 Efficient Frontier Visualization

In [ ]:
# Create efficient frontier plot
plt.figure(figsize=(12, 8))

# Plot all portfolios
plt.scatter(results[1], results[0], c=results[2], cmap='viridis', alpha=0.6)
plt.colorbar(label='Sharpe Ratio')

# Highlight optimal portfolios
plt.scatter(max_sharpe_volatility, max_sharpe_return, 
           marker='*', color='red', s=500, label='Maximum Sharpe Ratio')
plt.scatter(min_vol_volatility, min_vol_return, 
           marker='*', color='blue', s=500, label='Minimum Volatility')

# Plot individual assets
for i, asset in enumerate(assets):
    asset_return = expected_returns_vector[asset]
    asset_vol = np.sqrt(cov_matrix.loc[asset, asset])
    plt.scatter(asset_vol, asset_return, marker='o', s=200, label=asset)

plt.title('Efficient Frontier', fontsize=16, fontweight='bold')
plt.xlabel('Volatility (Risk)')
plt.ylabel('Expected Return')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Display optimal portfolio details
print("\nOPTIMAL PORTFOLIOS")
print("=" * 50)

print("\nMaximum Sharpe Ratio Portfolio:")
print(f"Expected Return: {max_sharpe_return:.4f} ({max_sharpe_return*100:.2f}%)")
print(f"Volatility: {max_sharpe_volatility:.4f} ({max_sharpe_volatility*100:.2f}%)")
print(f"Sharpe Ratio: {max_sharpe_ratio:.4f}")
print("Weights:")
for i, asset in enumerate(assets):
    print(f"  {asset}: {max_sharpe_weights[i]:.4f} ({max_sharpe_weights[i]*100:.2f}%)")

print("\nMinimum Volatility Portfolio:")
print(f"Expected Return: {min_vol_return:.4f} ({min_vol_return*100:.2f}%)")
print(f"Volatility: {min_vol_volatility:.4f} ({min_vol_volatility*100:.2f}%)")
print(f"Sharpe Ratio: {min_vol_ratio:.4f}")
print("Weights:")
for i, asset in enumerate(assets):
    print(f"  {asset}: {min_vol_weights[i]:.4f} ({min_vol_weights[i]*100:.2f}%)")

### 4.4 Portfolio Recommendation

In [ ]:
# Select recommended portfolio (Maximum Sharpe Ratio)
recommended_weights = max_sharpe_weights
recommended_return = max_sharpe_return
recommended_volatility = max_sharpe_volatility
recommended_sharpe = max_sharpe_ratio

print("PORTFOLIO RECOMMENDATION")
print("=" * 50)
print("\nRecommended Portfolio: Maximum Sharpe Ratio Portfolio")
print("\nRationale:")
print("- Maximizes risk-adjusted returns")
print("- Optimal balance between return and risk")
print("- Suitable for investors seeking efficient risk-return trade-off")

print("\nPortfolio Composition:")
portfolio_df = pd.DataFrame({
    'Asset': assets,
    'Weight': recommended_weights,
    'Weight (%)': recommended_weights * 100,
    'Expected Return': expected_returns_vector.values,
    'Expected Return (%)': expected_returns_vector.values * 100
})

print(portfolio_df.to_string(index=False, float_format='%.4f'))

print(f"\nPortfolio Metrics:")
print(f"Expected Annual Return: {recommended_return:.4f} ({recommended_return*100:.2f}%)")
print(f"Annual Volatility: {recommended_volatility:.4f} ({recommended_volatility*100:.2f}%)")
print(f"Sharpe Ratio: {recommended_sharpe:.4f}")

# Create portfolio composition pie chart
plt.figure(figsize=(10, 8))
colors = ['#ff9999', '#66b3ff', '#99ff99']
plt.pie(recommended_weights, labels=assets, autopct='%1.1f%%', 
        colors=colors, startangle=90)
plt.title('Recommended Portfolio Composition', fontsize=16, fontweight='bold')
plt.axis('equal')
plt.tight_layout()
plt.show()

## Task 5: Strategy Backtesting

### 5.1 Backtesting Setup

In [ ]:
# Define backtesting period (last year of data)
backtest_start = '2024-08-01'
backtest_end = '2025-07-31'

# Get backtesting data
backtest_prices = price_data[price_data.index >= backtest_start].copy()
backtest_returns = returns[returns.index >= backtest_start].copy()

print(f"Backtesting period: {backtest_start} to {backtest_end}")
print(f"Backtesting data points: {len(backtest_prices)}")

# Define portfolios
# Strategy portfolio: Our optimized portfolio
strategy_weights = pd.Series(recommended_weights, index=assets)

# Benchmark portfolio: 60% SPY, 40% BND
benchmark_weights = pd.Series([0.0, 0.4, 0.6], index=assets)  # [TSLA, BND, SPY]

print("\nPortfolio Weights:")
print("Strategy Portfolio:")
for asset, weight in strategy_weights.items():
    print(f"  {asset}: {weight:.4f} ({weight*100:.2f}%)")

print("\nBenchmark Portfolio (60/40 SPY/BND):")
for asset, weight in benchmark_weights.items():
    print(f"  {asset}: {weight:.4f} ({weight*100:.2f}%)")

### 5.2 Portfolio Performance Simulation

In [ ]:
# Calculate portfolio returns
def calculate_portfolio_returns(returns_df, weights):
    """
    Calculate portfolio returns given asset returns and weights
    """
    return (returns_df * weights).sum(axis=1)

# Calculate daily portfolio returns
strategy_returns = calculate_portfolio_returns(backtest_returns, strategy_weights)
benchmark_returns = calculate_portfolio_returns(backtest_returns, benchmark_weights)

# Calculate cumulative returns
strategy_cumulative = (1 + strategy_returns).cumprod()
benchmark_cumulative = (1 + benchmark_returns).cumprod()

# Calculate performance metrics
def calculate_performance_metrics(returns_series, risk_free_rate=0.02):
    """
    Calculate comprehensive performance metrics
    """
    # Annualized return
    total_return = (1 + returns_series).prod() - 1
    annualized_return = (1 + total_return) ** (252 / len(returns_series)) - 1
    
    # Annualized volatility
    annualized_volatility = returns_series.std() * np.sqrt(252)
    
    # Sharpe ratio
    excess_returns = returns_series - risk_free_rate / 252
    sharpe_ratio = excess_returns.mean() / returns_series.std() * np.sqrt(252)
    
    # Maximum drawdown
    cumulative = (1 + returns_series).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdown = (cumulative - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    
    return {
        'Total Return': total_return,
        'Annualized Return': annualized_return,
        'Annualized Volatility': annualized_volatility,
        'Sharpe Ratio': sharpe_ratio,
        'Maximum Drawdown': max_drawdown
    }

# Calculate metrics for both portfolios
strategy_metrics = calculate_performance_metrics(strategy_returns)
benchmark_metrics = calculate_performance_metrics(benchmark_returns)

print("BACKTESTING RESULTS")
print("=" * 50)

# Create comparison table
comparison_df = pd.DataFrame({
    'Strategy Portfolio': strategy_metrics,
    'Benchmark Portfolio': benchmark_metrics
})

print(comparison_df.round(4))

### 5.3 Performance Visualization

In [ ]:
# Create comprehensive performance visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Portfolio Backtesting Results', fontsize=16, fontweight='bold')

# 1. Cumulative Returns
ax1 = axes[0, 0]
ax1.plot(strategy_cumulative.index, strategy_cumulative, 
         label='Strategy Portfolio', linewidth=2, color='blue')
ax1.plot(benchmark_cumulative.index, benchmark_cumulative, 
         label='Benchmark Portfolio', linewidth=2, color='red')
ax1.set_title('Cumulative Returns')
ax1.set_ylabel('Cumulative Return')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Rolling Sharpe Ratio (30-day)
ax2 = axes[0, 1]
strategy_rolling_sharpe = strategy_returns.rolling(30).mean() / strategy_returns.rolling(30).std() * np.sqrt(252)
benchmark_rolling_sharpe = benchmark_returns.rolling(30).mean() / benchmark_returns.rolling(30).std() * np.sqrt(252)

ax2.plot(strategy_rolling_sharpe.index, strategy_rolling_sharpe, 
         label='Strategy Portfolio', linewidth=2, color='blue')
ax2.plot(benchmark_rolling_sharpe.index, benchmark_rolling_sharpe, 
         label='Benchmark Portfolio', linewidth=2, color='red')
ax2.set_title('Rolling Sharpe Ratio (30-day)')
ax2.set_ylabel('Sharpe Ratio')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Drawdown Analysis
ax3 = axes[1, 0]
strategy_cumulative_max = strategy_cumulative.expanding().max()
strategy_drawdown = (strategy_cumulative - strategy_cumulative_max) / strategy_cumulative_max

benchmark_cumulative_max = benchmark_cumulative.expanding().max()
benchmark_drawdown = (benchmark_cumulative - benchmark_cumulative_max) / benchmark_cumulative_max

ax3.fill_between(strategy_drawdown.index, strategy_drawdown, 0, 
                alpha=0.7, color='blue', label='Strategy Portfolio')
ax3.fill_between(benchmark_drawdown.index, benchmark_drawdown, 0, 
                alpha=0.7, color='red', label='Benchmark Portfolio')
ax3.set_title('Drawdown Analysis')
ax3.set_ylabel('Drawdown')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Monthly Returns Distribution
ax4 = axes[1, 1]
strategy_monthly = strategy_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)
benchmark_monthly = benchmark_returns.resample('M').apply(lambda x: (1 + x).prod() - 1)

ax4.hist(strategy_monthly, bins=10, alpha=0.7, label='Strategy Portfolio', color='blue')
ax4.hist(benchmark_monthly, bins=10, alpha=0.7, label='Benchmark Portfolio', color='red')
ax4.set_title('Monthly Returns Distribution')
ax4.set_xlabel('Monthly Return')
ax4.set_ylabel('Frequency')
ax4.legend()
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 5.4 Backtesting Conclusions

In [ ]:
# Performance comparison and conclusions
strategy_outperformed = strategy_metrics['Total Return'] > benchmark_metrics['Total Return']
strategy_better_sharpe = strategy_metrics['Sharpe Ratio'] > benchmark_metrics['Sharpe Ratio']

print("BACKTESTING CONCLUSIONS")
print("=" * 50)

print(f"\nStrategy vs Benchmark Performance:")
print(f"Total Return: Strategy {strategy_metrics['Total Return']:.4f} vs Benchmark {benchmark_metrics['Total Return']:.4f}")
print(f"Sharpe Ratio: Strategy {strategy_metrics['Sharpe Ratio']:.4f} vs Benchmark {benchmark_metrics['Sharpe Ratio']:.4f}")
print(f"Max Drawdown: Strategy {strategy_metrics['Maximum Drawdown']:.4f} vs Benchmark {benchmark_metrics['Maximum Drawdown']:.4f}")

print(f"\nOutperformance Analysis:")
if strategy_outperformed:
    outperformance = (strategy_metrics['Total Return'] - benchmark_metrics['Total Return']) * 100
    print(f"✓ Strategy outperformed benchmark by {outperformance:.2f} percentage points")
else:
    underperformance = (benchmark_metrics['Total Return'] - strategy_metrics['Total Return']) * 100
    print(f"✗ Strategy underperformed benchmark by {underperformance:.2f} percentage points")

if strategy_better_sharpe:
    print(f"✓ Strategy achieved better risk-adjusted returns (higher Sharpe ratio)")
else:
    print(f"✗ Benchmark achieved better risk-adjusted returns (higher Sharpe ratio)")

print(f"\nKey Insights:")
print(f"1. Model-driven approach {'validated' if strategy_outperformed else 'challenged'} by backtesting results")
print(f"2. TSLA forecast {'contributed positively' if strategy_outperformed else 'may have introduced excess risk'} to portfolio performance")
print(f"3. Diversification across asset classes {'effectively managed' if strategy_metrics['Maximum Drawdown'] > benchmark_metrics['Maximum Drawdown'] else 'helped manage'} downside risk")
print(f"4. {'Higher' if strategy_metrics['Annualized Volatility'] > benchmark_metrics['Annualized Volatility'] else 'Lower'} volatility suggests {'more aggressive' if strategy_metrics['Annualized Volatility'] > benchmark_metrics['Annualized Volatility'] else 'more conservative'} risk profile")

print(f"\nRecommendations:")
if strategy_outperformed:
    print("• Continue with model-driven approach but implement regular rebalancing")
    print("• Monitor forecast accuracy and adjust weights accordingly")
    print("• Consider expanding to more assets for better diversification")
else:
    print("• Review forecasting model parameters and methodology")
    print("• Consider reducing allocation to high-volatility assets like TSLA")
    print("• Implement more frequent rebalancing to capture changing market conditions")

print("\nLimitations of this backtest:")
print("• Single year testing period may not capture all market conditions")
print("• Transaction costs and slippage not considered")
print("• Model was not retrained during the backtesting period")
print("• Survivorship bias - only tested on assets that continued trading")

## Executive Summary and Investment Memo

### Final Recommendations for GMF Investments

In [ ]:
# Generate final investment memo summary
print("="*80)
print("GMF INVESTMENTS - INVESTMENT MEMO")
print("Time Series Forecasting for Portfolio Optimization")
print(f"Analysis Date: {datetime.now().strftime('%B %d, %Y')}")
print("="*80)

print("\nEXECUTIVE SUMMARY")
print("-"*50)
print("Our analysis employed advanced time series forecasting models (ARIMA and LSTM)")
print("to predict Tesla (TSLA) price movements and optimize portfolio allocation across")
print("three key assets: TSLA, BND (bonds), and SPY (broad market).")

print(f"\nKEY FINDINGS")
print("-"*50)
print(f"• Best performing model: {best_model} (RMSE: {min(arima_rmse, lstm_rmse):.2f})")
print(f"• 12-month TSLA forecast: ${forecast_end_price:.2f} ({total_return:+.1f}% expected return)")
print(f"• Optimal portfolio Sharpe ratio: {recommended_sharpe:.3f}")
print(f"• Backtesting {'validated' if strategy_outperformed else 'challenged'} our approach")

print(f"\nRECOMMENDED PORTFOLIO ALLOCATION")
print("-"*50)
for i, asset in enumerate(assets):
    print(f"• {asset}: {recommended_weights[i]*100:.1f}%")

print(f"\nEXPECTED PORTFOLIO METRICS")
print("-"*50)
print(f"• Expected Annual Return: {recommended_return*100:.2f}%")
print(f"• Expected Annual Volatility: {recommended_volatility*100:.2f}%")
print(f"• Expected Sharpe Ratio: {recommended_sharpe:.3f}")

print(f"\nRISK CONSIDERATIONS")
print("-"*50)
print("• High allocation to TSLA increases portfolio volatility")
print("• Forecast uncertainty increases significantly over longer horizons")
print("• Model performance may vary under different market conditions")
print("• Regular rebalancing recommended to maintain optimal allocation")

print(f"\nIMPLEMENTATION RECOMMENDATIONS")
print("-"*50)
print("1. Implement recommended allocation with monthly rebalancing")
print("2. Monitor model performance and retrain quarterly")
print("3. Set stop-loss limits for individual positions")
print("4. Regular review of correlation assumptions")
print("5. Consider expanding universe to include more asset classes")

print("\n" + "="*80)
print("END OF ANALYSIS")
print("="*80)

## Appendix: Model Diagnostics and Additional Analysis

In [ ]:
# Additional diagnostic plots and analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Model Diagnostics and Additional Analysis', fontsize=16, fontweight='bold')

# 1. Residual analysis for ARIMA
ax1 = axes[0, 0]
residuals = arima_fitted.resid
ax1.plot(residuals)
ax1.set_title('ARIMA Model Residuals')
ax1.set_ylabel('Residuals')
ax1.grid(True, alpha=0.3)

# 2. Q-Q plot for residuals
ax2 = axes[0, 1]
stats.probplot(residuals.dropna(), dist="norm", plot=ax2)
ax2.set_title('Q-Q Plot of Residuals')
ax2.grid(True, alpha=0.3)

# 3. Asset correlation over time
ax3 = axes[1, 0]
rolling_corr = returns['TSLA'].rolling(60).corr(returns['SPY'])
ax3.plot(rolling_corr.index, rolling_corr)
ax3.set_title('TSLA-SPY Rolling Correlation (60-day)')
ax3.set_ylabel('Correlation')
ax3.grid(True, alpha=0.3)

# 4. Volatility clustering
ax4 = axes[1, 1]
ax4.plot(returns['TSLA'].index, returns['TSLA']**2)
ax4.set_title('TSLA Squared Returns (Volatility Clustering)')
ax4.set_ylabel('Squared Returns')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Final model statistics
print("\nMODEL VALIDATION STATISTICS")
print("="*50)
print(f"ARIMA Model AIC: {arima_fitted.aic:.2f}")
print(f"ARIMA Model BIC: {arima_fitted.bic:.2f}")
print(f"Ljung-Box Test p-value: {arima_fitted.test_serial_correlation('ljungbox')[0]['lb_pvalue'].iloc[-1]:.4f}")

# Save key results for reporting
results_summary = {
    'best_model': best_model,
    'forecast_return': total_return,
    'optimal_weights': dict(zip(assets, recommended_weights)),
    'portfolio_metrics': {
        'return': recommended_return,
        'volatility': recommended_volatility,
        'sharpe': recommended_sharpe
    },
    'backtest_performance': strategy_outperformed
}

print("\nAnalysis completed successfully!")
print("All results have been generated and are ready for reporting.")